In [1]:
%env KAGGLE_USERNAME=despairbin
%env KAGGLE_KEY=8f465288dc1d4006f3c16609a9eea1d5
%env TOKENIZERS_PARALLELISM=true

env: KAGGLE_USERNAME=despairbin
env: KAGGLE_KEY=8f465288dc1d4006f3c16609a9eea1d5
env: TOKENIZERS_PARALLELISM=true


In [2]:
# 从 Kaggle 下载电影数据集
import kaggle
kaggle.api.authenticate()
kaggle.api.dataset_download_files('rounakbanik/the-movies-dataset', path='dataset', unzip=True)

In [3]:
# 数据集下载完成后，使用 pandas 的 read_csv()读取数据集数据
import pandas
import pandas as pd
movies=pd.read_csv('dataset/movies_metadata.csv',low_memory=False)
movies.shape

(45466, 24)

In [4]:
# 查看所有列的信息
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [6]:
# 筛选我们需要的列，并显示前5行
trimmed_movies = movies[["id", "title", "overview", "release_date", "genres"]]
trimmed_movies.head(5)

,id,title,overview,release_date,genres
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,1995-12-15,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,1995-12-22,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,1995-02-10,"[{'id': 35, 'name': 'Comedy'}]"


In [7]:
# 将数据转换为字典形式，清洗数据，确保电影的必要属性不为空，并将符合条件的电影加入到 movies_dict 列表中
unclean_movies_dict = trimmed_movies.to_dict('records')
print('{} movies'.format(len(unclean_movies_dict)))
movies_dict = []

for movie in unclean_movies_dict:
    if  movie["overview"] == movie["overview"] and movie["release_date"] == movie["release_date"] and movie["genres"] == movie["genres"] and movie["title"] == movie["title"]:
        movies_dict.append(movie)

45466 movies


In [8]:
# 连接到 milvus
from pymilvus import *
milvus_uri="tcp://127.0.0.1:19530"
token="dff2c39f81cfa34a851cb901a2da3a8d9a8bda3d30d17e1151c2e887c0bc3021703e41d48a89f1aaedd6c15ab56a30eb675b0e00"
connections.connect("default", uri=milvus_uri, token=token)
print("Connected!")

Connected!


In [9]:
# 将电影信息转化为 Embedding 向量
COLLECTION_NAME = 'film_vectors'
PARTITION_NAME = 'Movie'

id = FieldSchema(name='title', dtype=DataType.VARCHAR, max_length=500, is_primary=True)
field = FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, dim=384)
schema = CollectionSchema(fields=[id, field], description="movie recommender: film vectors", enable_dynamic_field=True)

if utility.has_collection(COLLECTION_NAME): # drop the same collection created before
    collection = Collection(COLLECTION_NAME)
    collection.drop()
collection = Collection(name=COLLECTION_NAME, schema=schema)
print("Collection created.")
index_params = {"index_type": "IVF_FLAT","metric_type": "L2","params": {"nlist": 128},
}
collection.create_index(field_name="embedding", index_params=index_params)
collection.load()

print("Collection indexed!")

Collection created.
Collection indexed!


In [10]:
# 用 SentenceTransformer 生成向量
from sentence_transformers import SentenceTransformer
import ast

# 定义函数从电影的 genre 列中提取文本
def build_genres(data):
    genres = data['genres']
    genre_list = ""
    entries= ast.literal_eval(genres)
    genres = ""
    for entry in entries:
        genre_list = genre_list + entry["name"] + ", "
    genres += genre_list
    genres = "".join(genres.rsplit(",", 1))
    return genres

# 创建 SentenceTransformer
transformer = SentenceTransformer('all-MiniLM-L6-v2')

def embed_movie(data):
    embed = "{} Released on {}. Genres are {}.".format(data["overview"], data["release_date"], build_genres(data))
    embeddings = transformer.encode(embed)
    return embeddings


In [11]:
# 将向量导入 Milvu
j = 0 
batch_size = 5
batch = []

for movie_dict in movies_dict:
    try:
        movie_dict["embedding"] = embed_movie(movie_dict)
        batch.append(movie_dict)
        j += 1
        if j % batch_size == 0:
            print("Embedded {} records".format(j))
            collection.insert(batch)
            print("Batch insert completed")
            batch = []
    except Exception as e:
        print("Error inserting record {}".format(e))
        print(batch)
        break

# 插入剩余记录
for movie_dict in batch:
    try:
        collection.insert([movie_dict])  # 插入单个向量
        j += 1
    except Exception as e:
        print("Error inserting record {}".format(e))
        print(movie_dict)

print("Final batch completed")
print("Finished with {} embeddings".format(j))


Embedded 5 records
Batch insert completed
Embedded 10 records
Batch insert completed
Embedded 15 records
Batch insert completed
Embedded 20 records
Batch insert completed
Embedded 25 records
Batch insert completed
Embedded 30 records
Batch insert completed
Embedded 35 records
Batch insert completed
Embedded 40 records
Batch insert completed
Embedded 45 records
Batch insert completed
Embedded 50 records
Batch insert completed
Embedded 55 records
Batch insert completed
Embedded 60 records
Batch insert completed
Embedded 65 records
Batch insert completed
Embedded 70 records
Batch insert completed
Embedded 75 records
Batch insert completed
Embedded 80 records
Batch insert completed
Embedded 85 records
Batch insert completed
Embedded 90 records
Batch insert completed
Embedded 95 records
Batch insert completed
Embedded 100 records
Batch insert completed
Embedded 105 records
Batch insert completed
Embedded 110 records
Batch insert completed
Embedded 115 records
Batch insert completed
Embedded

In [12]:
# 加载集合内存以准备搜索
collection.load() 

topK = 5
SEARCH_PARAM = {
    "metric_type":"L2",
    "params":{"nprobe": 20},
}
# 将搜索字符串转换为 embeddings
def embed_search(search_string):
    search_embeddings = transformer.encode(search_string)
    return search_embeddings

# 搜索用户查询的相似电影 embeddings
def search_for_movies(search_string):
    user_vector = embed_search(search_string)
    return collection.search([user_vector],"embedding",param=SEARCH_PARAM, limit=topK, expr=None, output_fields=['title', 'overview'])

In [13]:
from pprint import pprint
# 测试搜索功能
search_string = "A comedy from the 1990s set in a hospital. The main characters are in their 20s and are trying to stop a vampire."
results = search_for_movies(search_string)

# 打印搜索结果
for hit in results[0]: 
    print(hit.entity.get('title'))
    print(hit.entity.get('overview'))
    print("-------------------------------")


Tainted
A sarcastic comedy thriller about a vampire who sets out to infect the blood of a hospital and the twenty-something slackers who set out to stop him.
-------------------------------
An American Vampire Story
A group of friends go on a vacation with some new friends that turn out to be vampires, they hire a famous vampire killer to help them.
-------------------------------
Not of This Earth
A science fiction vampire movie. The Vampire is an emissary from an embattled world near destruction who teleports to Earth to see if they can live here. He finds that our blood is nourishing and that at least one source of it is a steady stream of transfusions. He hypnotizes a Dr. to provide them and has his blond nurse move in to administer them.
-------------------------------
Therapy for a Vampire
Horror comedy film following vampire count Geza von Kösznöm who's visiting groundbreaking neurologist Sigund Freud because he's bored of his life and frustrated of the "eternally long" relation